## 1 - Import dependencies

**Using**

- sbol3: package with sbol3 object constructors
- sbol-utilities: has simpler, straightforward functions to add parts
- tyto: to lookup ontology names

In [1]:
from sbol3 import *
from sbol_utilities.calculate_sequences import compute_sequence
from sbol_utilities.component import *
from sbol_utilities.helper_functions import url_to_identity
import tyto

## 2 - Set namespace and document

In [2]:
set_namespace('https://synbiohub.org/public/igem/')
doc = Document()

We're trying to create a minimal expression cassette device, with an rbs, cds and terminator:

![basic_cassette](iwbda_minimal_noslides-1.png "Basic expression cassette")

This does not have a promoter because that will be added later on to showcase a different functionality



## 3 - Create device component

In [3]:
i13504 = Component('i13504', SBO_DNA)
i13504.name = 'iGEM 2016 interlab reporter'
i13504.description = 'GFP expression cassette used for 2016 iGEM interlab study'
i13504.roles.append(tyto.SO.engineered_region)

Add the GFP expression cassette to the document. Notice that the object added is also returned, so this can be used as a pass-through call.


In [4]:
doc.add(i13504)

Right now we have an object with a name and a description, but no parts or sequences.

Let's create each part and add them to the document

## 4 - Create individual parts of the cassette

Here we will create a part-subpart hierarchy. We will also start using [SBOL-Utilities](https://github.com/synbiodex/sbol-utilities) to make it easier to create parts and to assemble those parts into a hierarchy.

![Cassette with parts](iwbda_minimal_noslides-2.png)

First, create the **RBS component**...

In [5]:
b0034, b0034_seq = doc.add(rbs('B0034', sequence='aaagaggagaaa', name='RBS (Elowitz 1999)'))

Next, create the **CDS component**

In [6]:
e0040_sequence = 'atgcgtaaaggagaagaacttttcactggagttgtcccaattcttgttgaattagatggtgatgttaatgggcacaaattttctgtcagtggagagggtgaaggtgatgcaacatacggaaaacttacccttaaatttatttgcactactggaaaactacctgttccatggccaacacttgtcactactttcggttatggtgttcaatgctttgcgagatacccagatcatatgaaacagcatgactttttcaagagtgccatgcccgaaggttatgtacaggaaagaactatatttttcaaagatgacgggaactacaagacacgtgctgaagtcaagtttgaaggtgatacccttgttaatagaatcgagttaaaaggtattgattttaaagaagatggaaacattcttggacacaaattggaatacaactataactcacacaatgtatacatcatggcagacaaacaaaagaatggaatcaaagttaacttcaaaattagacacaacattgaagatggaagcgttcaactagcagaccattatcaacaaaatactccaattggcgatggccctgtccttttaccagacaaccattacctgtccacacaatctgccctttcgaaagatcccaacgaaaagagagaccacatggtccttcttgagtttgtaacagctgctgggattacacatggcatggatgaactatacaaataataa'
e0040, _ = doc.add(cds('E0040', sequence=e0040_sequence, name='GFP'))

Finally, create the **Terminator**

In [7]:
b0015_sequence = 'ccaggcatcaaataaaacgaaaggctcagtcgaaagactgggcctttcgttttatctgttgtttgtcggtgaacgctctctactagagtcacactggctcaccttcgggtgggcctttctgcgtttata'
b0015, _ = doc.add(terminator('B0015', sequence=b0015_sequence, name='double terminator'))

Now construct the part-subpart hierarchy and order the parts: RBS before CDS, CDS before terminator

In [8]:
order(b0034, e0040, i13504)
order(e0040, b0015, i13504)

Now we have a document with objects for our device and its parts, which are listed as subcomponents of that device. We also have constraints that say the RBS goes before the CDS, and the CDS goes before the Terminator.

However, we still can't get the entire sequence from the device object alone:

In [9]:
i13504.sequences

[]

## 5 - Add SubComponent locations

Here we add base coordinates to SubComponents:

![SubC coordinates](iwbda_minimal_noslides-3.png)

But first, use `compute_sequence` to get the full sequence for the BBa_I13504 device

See http://parts.igem.org/Part:BBa_I13504

In [10]:
i13504_seq = compute_sequence(i13504)

In [11]:
i13504_seq.elements

'aaagaggagaaaatgcgtaaaggagaagaacttttcactggagttgtcccaattcttgttgaattagatggtgatgttaatgggcacaaattttctgtcagtggagagggtgaaggtgatgcaacatacggaaaacttacccttaaatttatttgcactactggaaaactacctgttccatggccaacacttgtcactactttcggttatggtgttcaatgctttgcgagatacccagatcatatgaaacagcatgactttttcaagagtgccatgcccgaaggttatgtacaggaaagaactatatttttcaaagatgacgggaactacaagacacgtgctgaagtcaagtttgaaggtgatacccttgttaatagaatcgagttaaaaggtattgattttaaagaagatggaaacattcttggacacaaattggaatacaactataactcacacaatgtatacatcatggcagacaaacaaaagaatggaatcaaagttaacttcaaaattagacacaacattgaagatggaagcgttcaactagcagaccattatcaacaaaatactccaattggcgatggccctgtccttttaccagacaaccattacctgtccacacaatctgccctttcgaaagatcccaacgaaaagagagaccacatggtccttcttgagtttgtaacagctgctgggattacacatggcatggatgaactatacaaataataaccaggcatcaaataaaacgaaaggctcagtcgaaagactgggcctttcgttttatctgttgtttgtcggtgaacgctctctactagagtcacactggctcaccttcgggtgggcctttctgcgtttata'

`compute_sequence` added `Range`s to the subcomponents. Let's inspect our objects and check one of those ranges to see that the values are what we expect.

The expected range of the terminator is (733, 861).

In [12]:
for feature in i13504.features:
    print(feature.instance_of)

https://synbiohub.org/public/igem/B0034
https://synbiohub.org/public/igem/E0040
https://synbiohub.org/public/igem/B0015


In [13]:
i13504.features[2].locations

In [14]:
b0015_range = i13504.features[2].locations[0]
print(f'Range of {b0015.display_name}: ({b0015_range.start}, {b0015_range.end})')

Range of double terminator: (733, 861)
